# Optmiziation of Deep Learning for Cardiologist-level Myocardial Infarction Detection in Electrocardiograms

In [119]:
## Many neural networks we tested: AlexNet, ResNet50, Gradient Boosting CNN, ConvNetQuake
## the best one so far is the ConvQuakeNet

##start by processing the data
##then create the nn.Model subclass
##then call the Pso(forward function of model)
##then return loss score; ie optimize hyperparameters for search convergence

In [2]:
##extract data in dataframes, use 2 channels, v6: anterior lead, vz: left wrist lead
##two most prominent in myocardial infarction diagnosis
## no cleaning required, data is clean
from __future__ import print_function
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import wfdb
import time
import random
from sklearn.preprocessing import minmax_scale
import sys
from torch.utils.tensorboard import SummaryWriter

E:\Downloads\ANACONDA3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\Downloads\ANACONDA3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
E:\Downloads\ANACONDA3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
E:\Downloads\ANACONDA3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
from torch.nn import Sequential, Linear, MSELoss
from torch_pso import ParticleSwarmOptimizer

In [4]:
torch.cuda.is_available()

False

In [5]:
torch.__version__

'2.2.1+cpu'

In [130]:
#pip install numpy --pre torch torchvision torchaudio --force-reinstall --index-url https://download.pytorch.org/whl/nightly/cu117

In [131]:
torch.cuda.is_available()

False

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD  ##use the 2 apis PyTorch and KerasCV, KerasCV is easier

In [7]:
##experiment with seed, run, channels(sinoatrial (SA) node, left feature lead, septal)
##later experiment with channel averaging of physician diagnosis features of myocardial infarction
##(septal average: v1, v2 ; left anterior average: v5, v6; left wrist: avl)
seed_num = 39 #sys.argv[1]  ##required bash files
#run_num = #sys.argv[2]
channel_1 = 'v6'#sys.argv[3]
channel_2 = 'vz'#sys.argv[4]
channel_3 = 'ii'#sys.argv[5]

In [8]:
with open('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/RECORDS') as fp:  
    lines = fp.readlines()

In [9]:
#lines  ##display lines in records file

In [10]:
files_myocardial = []
files_healthy = []

In [11]:
for file in lines:
    file_path = './ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1] + '.hea'
    
    ##read header to determine class ##focus on myocardial infarction
    if 'Myocardial infarction' in open(file_path).read():
        files_myocardial.append(file)
        
    if 'Healthy control' in open(file_path).read():
        files_healthy.append(file)

In [12]:
##shuffle data (cross-validation)
np.random.seed(int(seed_num))
np.random.shuffle(files_myocardial) 
np.random.shuffle(files_healthy)

In [13]:
files_healthy[0:5]

['patient240/s0468_re\n',
 'patient246/s0478_re\n',
 'patient150/s0287lre\n',
 'patient233/s0482_re\n',
 'patient244/s0473_re\n']

In [14]:
#files_healthy  ##check with CONTROL
#files_myocardial

In [15]:
##split to train and test, 80% train, 20% test
healthy_train = files_healthy[:int(0.8*len(files_healthy))]
healthy_val = files_healthy[int(0.8*len(files_healthy)):]
myocardial_train = files_myocardial[:int(0.8*len(files_myocardial))]
myocardial_val = files_myocardial[int(0.8*len(files_myocardial)):]

In [16]:
healthy_train[0:5]

['patient240/s0468_re\n',
 'patient246/s0478_re\n',
 'patient150/s0287lre\n',
 'patient233/s0482_re\n',
 'patient244/s0473_re\n']

In [17]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [18]:
patient_ids_healthy_train = []
patient_ids_healthy_val = []
patient_ids_myocardial_train = []
patient_ids_myocardial_val = []
##extract first 10 letters in file path string, which is the patient id
for index in healthy_train:
    patient_ids_healthy_train.append(index[0:10])
for index in healthy_val:
    patient_ids_healthy_val.append(index[0:10])
for index in myocardial_train:
    patient_ids_myocardial_train.append(index[0:10])
for index in myocardial_val:
    patient_ids_myocardial_val.append(index[0:10])

In [48]:
#type(patient_ids_myocardial_train)  ##list

list

In [19]:
len(np.unique(patient_ids_myocardial_train)) ##80% of total 148 patients is 118  
#likely to have duplicates, so distribute intersection over train and validation sets
#they can have common ecgs, used for checking

140

In [20]:
len(np.unique(patient_ids_myocardial_val)) ##20% of total 148 patients is 30

57

In [63]:
#patient_ids_myocardial_val##stable

In [21]:
intersection_myocardial = intersection(patient_ids_myocardial_train, patient_ids_myocardial_val)
intersection_healthy = intersection(patient_ids_healthy_train, patient_ids_healthy_val)

In [22]:
##myocardial 
move_to_train = intersection_myocardial[:int(0.5*len(intersection_myocardial))]
move_to_val = intersection_myocardial[int(0.5*len(intersection_myocardial)):]

In [23]:
move_to_train[0:5]

['patient223', 'patient017', 'patient100', 'patient080', 'patient045']

In [24]:
for patient_id in move_to_train:
    in_val = []
    ##find and remove all files in val ecg readings by id
    for file_ in myocardial_val:
        if file_[:10] == patient_id:
            in_val.append(file_)
            myocardial_val.remove(file_)
            
    ##add to train
    for file_ in in_val:
        myocardial_train.append(file_)

In [25]:
for patient_id in move_to_val:    
    in_train = []
    ##find and remove all files in train
    for file_ in myocardial_train:
        if file_[:10] == patient_id:
            in_train.append(file_)
            myocardial_train.remove(file_)
            
    ##add to val
    for file_ in in_train:
        myocardial_val.append(file_)

In [26]:
##healthy
move_to_train = intersection_healthy[:int(0.5*len(intersection_healthy))]
move_to_val = intersection_healthy[int(0.5*len(intersection_healthy)):]
print(move_to_train[0:5])

['patient251', 'patient245', 'patient233']


In [27]:
for patient_id in move_to_train:
    in_val = []
    ##find and remove all files in val ecg readings by id
    for file_ in healthy_val:
        if file_[:10] == patient_id:
            in_val.append(file_)
            healthy_val.remove(file_)
            
    ##add to train
    for file_ in in_val:
        healthy_train.append(file_)

In [28]:
for patient_id in move_to_val:    
    in_train = []
    ##find and remove all files in train
    for file_ in healthy_train:
        if file_[:10] == patient_id:
            in_train.append(file_)
            healthy_train.remove(file_)
            
    ##add to val
    for file_ in in_train:
        healthy_val.append(file_)

In [29]:
len(np.unique(myocardial_val))

98

In [30]:
healthy_train[0][:-1]

'patient240/s0468_re'

In [31]:
data_healthy_train = []
for file in healthy_train:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_healthy_train.append(data)

In [32]:
data_healthy_train[0]##array output of each of the three channels

[array([-0.1245, -0.1275, -0.1305, ...,  0.188 ,  0.187 ,  0.1855]),
 array([ 0.458 ,  0.458 ,  0.457 , ..., -0.335 , -0.3325, -0.329 ]),
 array([-0.1025, -0.1   , -0.0925, ...,  0.062 ,  0.0695,  0.065 ])]

In [33]:
data_healthy_val = []
for file in healthy_val:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_healthy_val.append(data)

In [34]:
data_healthy_val[0]

[array([-0.1595, -0.1605, -0.164 , ...,  0.106 ,  0.1025,  0.0995]),
 array([-0.008 , -0.007 , -0.0065, ..., -0.074 , -0.0725, -0.071 ]),
 array([-0.0795, -0.0805, -0.082 , ...,  0.046 ,  0.0445,  0.042 ])]

In [35]:
data_unhealthy_train = []
for file in myocardial_train:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_unhealthy_train.append(data)

In [36]:
data_unhealthy_val = []
for file in myocardial_val:
    ##records for each
    data_ii, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_1)])
    data_v6, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_2)])
    data_vz, _ = wfdb.rdsamp('./ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/' + file[:-1], channel_names=[str(channel_3)])
    data = [data_ii.flatten(), data_v6.flatten(), data_vz.flatten()]  ##flatten to input directly into keras model
    data_unhealthy_val.append(data)

In [37]:
##check they are all arrays
data_healthy_train = np.asarray(data_healthy_train)
data_healthy_val = np.asarray(data_healthy_val)
data_unhealthy_train = np.asarray(data_unhealthy_train)
data_unhealthy_val = np.asarray(data_unhealthy_val)

C:\Users\dell\AppData\Local\Temp\ipykernel_14992\2361096082.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_healthy_train = np.asarray(data_healthy_train)
C:\Users\dell\AppData\Local\Temp\ipykernel_14992\2361096082.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_healthy_val = np.asarray(data_healthy_val)
C:\Users\dell\AppData\Local\Temp\ipykernel_14992\2361096082.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes)

In [38]:
data_healthy_train[0] ##true

array([array([-0.1245, -0.1275, -0.1305, ...,  0.188 ,  0.187 ,  0.1855]),
       array([ 0.458 ,  0.458 ,  0.457 , ..., -0.335 , -0.3325, -0.329 ]),
       array([-0.1025, -0.1   , -0.0925, ...,  0.062 ,  0.0695,  0.065 ])],
      dtype=object)

In [39]:
window_size = 10000  ##from each ecg reading

In [40]:
def get_batch(batch_size, split='train'):
    ##random sampling of batches
    ##divide batch number in half to improve speed of network
    if split == 'train':
        unhealthy_indices = random.sample(sorted(np.arange(len(data_unhealthy_train))), k=int(batch_size / 2))
        healthy_indices = random.sample(sorted(np.arange(len(data_healthy_train))), k=int(batch_size / 2))
        unhealthy_batch = data_unhealthy_train[unhealthy_indices]
        healthy_batch = data_healthy_train[healthy_indices]
    elif split == 'val': 
        unhealthy_indices = random.sample(sorted(np.arange(len(data_unhealthy_val))), k=int(batch_size / 2))
        healthy_indices = random.sample(sorted(np.arange(len(data_healthy_val))), k=int(batch_size / 2))
        unhealthy_batch = data_unhealthy_val[unhealthy_indices]
        healthy_batch = data_healthy_val[healthy_indices]
    
    batch_x = []  ##batch of mixed healthy and unhealthy data
    for sample in unhealthy_batch: ##if val or if train
        start = random.choice(np.arange(len(sample[0]) - window_size))  ##randomly sample window from ecg 
        # normalize ecg values 
        normalized_1 = minmax_scale(sample[0][start:start+window_size])
        normalized_2 = minmax_scale(sample[1][start:start+window_size])
        normalized_3 = minmax_scale(sample[2][start:start+window_size])
        normalized = np.array((normalized_1, normalized_2, normalized_3))
        batch_x.append(normalized)
        
    for sample in healthy_batch:
        start = random.choice(np.arange(len(sample[0]) - window_size))
        # normalize
        normalized_1 = minmax_scale(sample[0][start:start+window_size])
        normalized_2 = minmax_scale(sample[1][start:start+window_size])
        normalized_3 = minmax_scale(sample[2][start:start+window_size])
        normalized = np.array((normalized_1, normalized_2, normalized_3))
        batch_x.append(normalized)
    
    batch_y = [0.1 for _ in range(int(batch_size / 2))]
    for _ in range(int(batch_size / 2)):
        batch_y.append(0.9)
        
    indices = np.arange(len(batch_y))
    np.random.shuffle(indices)
    
    batch_x = np.array(batch_x)
    batch_y = np.array(batch_y)
    
    batch_x = batch_x[indices]
    batch_y = batch_y[indices]
    
    batch_x = np.reshape(batch_x, (-1, 3, window_size))
    batch_x = torch.from_numpy(batch_x)
    batch_x = batch_x.float()#.cuda()
    batch_x = batch_x.float()
    
    batch_y = np.reshape(batch_y, (-1, 1))
    batch_y = torch.from_numpy(batch_y)
    batch_y = batch_y.float()#.cuda()
    batch_y = batch_y.float()
    
    return batch_x, batch_y

In [41]:
##keras sequential model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import copy

In [42]:
class ConvNetQuake(nn.Module):##issues with keras attributes, use PyTorch
    def __init__(self):
        super(ConvNetQuake, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv6 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv7 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.conv8 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.linear1 = nn.Linear(1280, 128)
        self.linear2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()
        self.bn1 = nn.BatchNorm1d(32)
        self.bn2 = nn.BatchNorm1d(32)
        self.bn3 = nn.BatchNorm1d(32)
        self.bn4 = nn.BatchNorm1d(32)
        self.bn5 = nn.BatchNorm1d(32)
        self.bn6 = nn.BatchNorm1d(32)
        self.bn7 = nn.BatchNorm1d(32)
        self.bn8 = nn.BatchNorm1d(32)
        '''self.conv1 = keras.layers.Conv1D(kernel_size=3, stride=2, padding=1, data_format=(batch, features, steps))(x)
        self.dense2 = keras.layers.Dense(5, activation="softmax")
        self.dropout = keras.layers.Dropout(0.5)'''

    def forward(self, x):  ##x is preliminary input
        x = self.bn1(F.relu((self.conv1(x))))
        x = self.bn2(F.relu((self.conv2(x))))
        x = self.bn3(F.relu((self.conv3(x))))
        x = self.bn4(F.relu((self.conv4(x))))
        x = self.bn5(F.relu((self.conv5(x))))
        x = self.bn6(F.relu((self.conv6(x))))
        x = self.bn7(F.relu((self.conv7(x))))
        x = self.bn8(F.relu((self.conv8(x))))
        x = torch.reshape(x, (10, -1))
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        '''x = self.dense1(inputs)
        x = self.dropout(x, training=training)
        return self.dense2(x)'''
        return x

In [ ]:
'''net = Sequential(Linear(10,100), Linear(100,100), Linear(100,10))
optim = ParticleSwarmOptimizer(net.parameters(),
                               inertial_weight=0.5,
                               num_particles=100,
                               max_param_value=1,
                               min_param_value=-1)
criterion = MSELoss()
target = torch.rand((10,)).round()

x = torch.rand((10,))
for _ in range(100):
    
    def closure():
        # Clear any grads from before the optimization step, since we will be changing the parameters
        optim.zero_grad()  
        return criterion(net(x), target)
    
    optim.step(closure)
    print('Prediciton', net(x))
    print('Target    ', target)'''

In [46]:
model = ConvNetQuake()
#model.cuda() ##compute engine device

model = nn.DataParallel(model, device_ids=[0])
#optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-4)
optimizer = ParticleSwarmOptimizer(model.parameters(),
                               inertial_weight=0.5,
                               num_particles=100,
                               max_param_value=1,
                               min_param_value=-1)
criterion = nn.BCELoss()
##run the model now
num_iters = 50000
batch_size = 10

acc_values = []
acc_values_train = []

for iters in range(num_iters):
    batch_x, batch_y = get_batch(batch_size, split='train')
    y_pred = model(batch_x)
    
    def closure():
        # Clear any grads from before the optimization step, since we will be changing the parameters
        optimizer.zero_grad()  
        return criterion(y_pred, batch_y)
    
    loss = criterion(y_pred, batch_y)
    #optimizer.zero_grad()
    loss.backward()
    optimizer.step(closure)

    # validation
    if iters%100 == 0 and iters != 0:

        print('Loss/train', loss, iters)

        with torch.no_grad():

            # test_set
            iterations = 100
            avg_acc = 0

            for _ in range(iterations):
                batch_x, batch_y = get_batch(batch_size, split='val')
                cleaned = model(batch_x)

                count = 0
                acc = 0
                for num in cleaned:
                    if int(torch.round(num)) == int(torch.round(batch_y[count])):
                        acc += 10
                    count += 1
                avg_acc += acc

            acc_values.append((avg_acc / iterations))
            print('Accuracy/val', (avg_acc / iterations), iters)

            # train_set
            iterations = 100
            avg_acc = 0

            for _ in range(iterations):
                batch_x, batch_y = get_batch(batch_size, split='train')
                cleaned = model(batch_x)

                count = 0
                acc = 0
                for num in cleaned:
                    if int(torch.round(num)) == int(torch.round(batch_y[count])):
                        acc += 10
                    count += 1
                avg_acc += acc

            acc_values_train.append((avg_acc / iterations))
            print('Accuracy/train', (avg_acc / iterations), iters)

Loss/train tensor(50., grad_fn=<BinaryCrossEntropyBackward0>) 100
Accuracy/val 48.1 100
Accuracy/train 48.3 100
Loss/train tensor(41.5681, grad_fn=<BinaryCrossEntropyBackward0>) 200
Accuracy/val 48.8 200
Accuracy/train 49.3 200
Loss/train tensor(58., grad_fn=<BinaryCrossEntropyBackward0>) 300
Accuracy/val 49.2 300
Accuracy/train 48.4 300
Loss/train tensor(45.3223, grad_fn=<BinaryCrossEntropyBackward0>) 400
Accuracy/val 47.3 400
Accuracy/train 48.0 400
Loss/train tensor(49.2593, grad_fn=<BinaryCrossEntropyBackward0>) 500
Accuracy/val 48.7 500
Accuracy/train 47.6 500
Loss/train tensor(41.1650, grad_fn=<BinaryCrossEntropyBackward0>) 600
Accuracy/val 48.3 600
Accuracy/train 47.2 600
Loss/train tensor(31.2326, grad_fn=<BinaryCrossEntropyBackward0>) 700
Accuracy/val 48.2 700
Accuracy/train 47.4 700
Loss/train tensor(41.7382, grad_fn=<BinaryCrossEntropyBackward0>) 800
Accuracy/val 48.7 800
Accuracy/train 47.2 800
Loss/train tensor(35.8350, grad_fn=<BinaryCrossEntropyBackward0>) 900
Accuracy/v

KeyboardInterrupt: 

## _______________________________________________________________________

In [145]:
##PSO Classes
##Particle object class
class Particle(object):
    '''Particle class for PSO: each search member is a particle'''

    '''Arguments:
        lower_bound (np.array): Vector of lower boundaries for particle dimensions.
        upper_bound (np.array): Vector of upper boundaries for particle dimensions.
        dimensions (int): Number of dimensions of the search space.
        objective function (function): Black-box function to evaluate.
'''
    def __init__(self, lower_bound, upper_bound, dimensions, objective_function):
        self.reset(dimensions, lower_bound, upper_bound, objective_function)

    def reset(self, dimensions, lower_bound, upper_bound, objective_function):
        '''Particle reset: allows for reset of a particle without reallocation'''
        position = []
        for i in range(dimensions):
            if lower_bound[i] < upper_bound[i]:
                position.extend(np.random.randint(lower_bound[i], upper_bound[i] + 1, 1, dtype=int))
            elif lower_bound[i] == upper_bound[i]:
                position.extend(np.array([lower_bound[i]], dtype=int))
            else:
                assert False

        self.position = [position]

        self.velocity = [np.multiply(np.random.rand(dimensions),
                                     (upper_bound - lower_bound)).astype(int)]

        self.best_position = self.position[:]

        self.function_value = [objective_function(self.best_position[-1])]
        self.best_function_value = self.function_value[:]
    
    def update_velocity(self, omega, phip, phig, best_swarm_position):
        '''Particle velocity update'''

        '''Args:
            omega (float): Velocity equation constant.
            phip (float): Velocity equation constant.
            phig (float): Velocity equation constant.
            best_swarm_position (np.array): Best global particle position.'''

        random_coefficient_p = np.random.uniform(size=np.asarray(self.position[-1]).shape)
        random_coefficient_g = np.random.uniform(size=np.asarray(self.position[-1]).shape)

        self.velocity.append(omega*np.asarray(self.velocity[-1])
                             + (phip*random_coefficient_p
                                * (np.asarray(self.best_position[-1])- np.asarray(self.position[-1])))
                             + (phig* random_coefficient_g
                             * (np.asarray(best_swarm_position)- np.asarray(self.position[-1]))))

        self.velocity[-1] = self.velocity[-1].astype(int)

    def update_position(self, lower_bound, upper_bound, objective_function):
        '''Particle position update, current position + Vd+1'''
                             
        new_position = self.position[-1] + self.velocity[-1]

        if np.array_equal(self.position[-1], new_position):
            self.function_value.append(self.function_value[-1])
        else:
            mark1 = new_position < lower_bound
            mark2 = new_position > upper_bound

            new_position[mark1] = lower_bound[mark1]
            new_position[mark2] = upper_bound[mark2]

            self.function_value.append(objective_function(self.position[-1]))

        self.position.append(new_position.tolist())

        if self.function_value[-1] < self.best_function_value[-1]:
            self.best_position.append(self.position[-1][:])
            self.best_function_value.append(self.function_value[-1])

In [146]:
class Pso(object):
    '''PSO wrapper

    This class contains the particles and provides an abstraction to hold all the context
    of the PSO algorithm

    Args:
        swarmsize (int): Number of particles in the swarm
        maxiter (int): Maximum number of generations the swarm will run'''

    
    def __init__(self, swarmsize=100, maxiter=100):
        self.max_generations = maxiter
        self.swarmsize = swarmsize

        self.omega = 0.5  ##start c1 and c2 at 0.5, balance exploration and exploitation
        self.phip = 0.5
        self.phig = 0.5

        self.minstep = 1e-4
        self.minfunc = 1e-4

        self.best_position = [None]
        self.best_function_value = [1]

        self.particles = []

        self.retired_particles = []

    def run(self, function, lower_bound, upper_bound, kwargs=None):
        '''Perform a particle swarm optimization (PSO)'''

        '''Args:
            objective_function (function): The function to be minimized.
            lower_bound (np.array): Vector of lower boundaries for particle dimensions.
            upper_bound (np.array): Vector of upper boundaries for particle dimensions.

        Returns:
            best_position (np.array): Best known position
            accuracy (float): Objective value at best_position
            :param kwargs:'''

        
        if kwargs is None:
            kwargs = {}

        objective_function = lambda x: function(x, **kwargs)
        assert hasattr(function, '__call__'), 'Invalid function handle'

        assert len(lower_bound) == len(upper_bound), 'Invalid bounds length'

        lower_bound = np.array(lower_bound)
        upper_bound = np.array(upper_bound)

        assert np.all(upper_bound > lower_bound), 'Invalid boundary values'


        dimensions = len(lower_bound)

        self.particles = self.initialize_particles(lower_bound,
                                                   upper_bound,
                                                   dimensions,
                                                   objective_function)

        #Start evolution
        generation = 1
        while generation <= self.max_generations:
            for particle in self.particles:
                particle.update_velocity(self.omega, self.phip, self.phig, self.best_position[-1])
                particle.update_position(lower_bound, upper_bound, objective_function)

                if particle.best_function_value[-1] == 0:
                    self.retired_particles.append(copy.deepcopy(particle))
                    particle.reset(dimensions, lower_bound, upper_bound, objective_function)
                elif particle.best_function_value[-1] < self.best_function_value[-1]:
                    stepsize = np.sqrt(np.sum((np.asarray(self.best_position[-1])
                                               - np.asarray(particle.position[-1])) ** 2))

                    if np.abs(np.asarray(self.best_function_value[-1])
                              - np.asarray(particle.best_function_value[-1])) \
                            <= self.minfunc:
                        return particle.best_position[-1], particle.best_function_value[-1]
                    elif stepsize <= self.minstep:
                        return particle.best_position[-1], particle.best_function_value[-1]
                    else:
                        self.best_function_value.append(particle.best_function_value[-1])
                        self.best_position.append(particle.best_position[-1][:])



            generation += 1

        return self.best_position[-1], self.best_function_value[-1]

    def initialize_particles(self, lower_bound, upper_bound, dimensions, objective_function):
        '''Initializes the particles for the swarm'''

        '''Args:
            objective_function (function): The function to be minimized.
            lower_bound (np.array): Vector of lower boundaries for particle dimensions.
            upper_bound (np.array): Vector of upper boundaries for particle dimensions.
            dimensions (int): Number of dimensions of the search space.

        Returns:
            particles (list): Collection or particles in the swarm'''
        
        particles = []
        for _ in range(self.swarmsize):
            particles.append(Particle(lower_bound,
                                      upper_bound,
                                      dimensions,
                                      objective_function))
            if particles[-1].best_function_value[-1] < self.best_function_value[-1]:
                self.best_function_value.append(particles[-1].best_function_value[-1])
                self.best_position.append(particles[-1].best_position[-1])


        self.best_position = [self.best_position[-1]]
        self.best_function_value = [self.best_function_value[-1]]

        return particles

In [147]:
##running model with optimization function
##1) define forward function
##2) call Pso with swarmsize = input shape of input attribute of forward function in model

'''model = ConvNetQuake()
#model.cuda() ##compute engine device

model = nn.DataParallel(model, device_ids=[0])

optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-4)
criterion = nn.BCELoss()'''

In [1]:
#pip install torch-pso  ##check with built in torch pso

In [149]:
'''num_iters = 50000
batch_size = 10

acc_values = []
acc_values_train = []

for iters in range(num_iters):

    batch_x, batch_y = get_batch(batch_size, split='train')  ##get train and test batches
    ###initialize PSO with forward input
    pso = Pso(swarmsize=batch_size,maxiter=num_iters)
    y_pred = model(batch_x)
    bp,value = pso.run(model(batch_x),[1,2,2,2],[16,8,4,4]) ##run model and get optimized outputs
    loss = criterion(y_pred, batch_y)
    ##bp === is loss
    ##value , v === is accuracy
    v = model(bp)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    ###
    #y_pred = model(batch_x) ##run model on input of healthy and unhealthy patients
    #loss = criterion(y_pred, batch_y)
    #optimizer.zero_grad()
    #loss.backward()
    #optimizer.step()
    # validation
    if iters%100 == 0 and iters != 0:

        print('Loss/train', loss, iters)
        print('PSO Loss/train', bp, iters)
        #writer.add_scalar('Loss/train', loss, iters)

        with torch.no_grad():

            # test_set
            iterations = 100
            avg_acc = 0

            for _ in range(iterations):
                batch_x, batch_y = get_batch(batch_size, split='val')
                cleaned = model(batch_x)

                count = 0
                acc = 0
                for num in cleaned:
                    if int(round(num)) == int(round(batch_y[count])):
                        acc += 10
                    count += 1
                avg_acc += acc

            acc_values.append((avg_acc / iterations))
            print('Accuracy/val', (avg_acc / iterations), iters)
            #writer.add_scalar('Accuracy/val', (avg_acc / iterations), iters)

            # train_set
            iterations = 100
            avg_acc = 0

            for _ in range(iterations):
                batch_x, batch_y = get_batch(batch_size, split='train')
                cleaned = model(batch_x)

                count = 0
                acc = 0
                for num in cleaned:
                    if int(round(num)) == int(round(batch_y[count])):
                        acc += 10
                    count += 1
                avg_acc += acc

            acc_values_train.append((avg_acc / iterations))
            #writer.add_scalar('Accuracy/train', (avg_acc / iterations), iters)
            print('Accuracy/train', (avg_acc / iterations), iters)'''

AssertionError: Invalid function handle

In [ ]:
'''def func(x):
  n,sf,sp,l = x[0],x[1],x[2],x[3]
 
  model = Sequential()
  model.add(Conv2D(32,kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(Conv2D(n, (sf, sf), activation='relu'))
  model.add(MaxPooling2D(pool_size=(sp, sp),strides=(l,l)))
  model.add(Flatten())
  model.add(Dense(num_classes, activation='softmax'))
  model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
  
  cp = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')];

  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=0,
            validation_data=(x_test, y_test),callbacks=cp)
  
  score = model.evaluate(x_test, y_test, verbose=0)

  # loss, val
  print('current config:',x,'val:',score[1])
  return score[1]

##################################################################
pso = Pso(swarmsize=4,maxiter=14)
# n,sf,sp,l
bp,value = pso.run(func,[1,2,2,2],[16,8,4,4])

v = func(bp);

##################################################################

print('Test loss:', bp)
print('Test accuracy:', value,v)'''

In [ ]:
#model = MyModel()